In [1]:
## Model is based upon the SEQ2SEQ model used in the Keras blog
## https://keras.io/examples/nlp/lstm_seq2seq/

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

c:\users\abala\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\abala\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\users\abala\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df = pd.read_csv('preprocessedDataset.csv')
df.head()
df = df.dropna()
df['speaker'] = df['speaker'].astype('str')
df['response'] = df['response'].astype('str')

In [4]:
def createSpeakerSet(text):
    global promptSet
    global promptDict
    
    text = str(text)
    for word in text.split(' '):
        if word in promptDict:
            promptDict[word] = promptDict[word] + 1
            if promptDict[word] > 2:
                promptSet.add(word)
        else:
            promptDict[word] = 1
        
def createResponseSet(text):
    global respnseSet
    global reponseDict
    
    text = str(text)
    for word in text.split(' '):
        if word in responseDict:
            responseDict[word] = responseDict[word] + 1
            if responseDict[word] > 2:
                responseSet.add(word)
        else:
            responseDict[word] = 1

In [5]:
promptDict = {}
responseDict = {}

promptSet = set()
responseSet = set()

## add the end of the line word
promptSet.add("<END>")
responseSet.add("<END>")

## add the word not in vocab word
promptSet.add("<OTHER>")
responseSet.add("<OTHER>")


## create a set of the vocab for the speaker and the response
_ = df['speaker'].apply(createSpeakerSet)
_ = df['response'].apply(createResponseSet)

## create the sorted vocabulary
promptVocab = sorted(list(promptSet))
responseVocab = sorted(list(promptSet))

## get the size of the vocab
promptVocabSize = len(promptVocab)
responseVocabSize = len(responseVocab)

## get the longest prompt and response
#maxLenPrompt = max([len(str(txt)) for txt in list(df['speaker'])])
maxLenPrompt = 50 + 1
#maxLenResponse = max([len(str(txt)) for txt in list(df['response'])])
maxLenResponse = 50 + 1


## create the empty input and output vectors
encoderInputSize = (len(df), maxLenPrompt, promptVocabSize)
encoder_input = np.zeros(encoderInputSize, dtype = "float32")

decoderInputSize = (len(df), maxLenResponse, responseVocabSize)
decoder_input = np.zeros(decoderInputSize, dtype = "float32")

decoder_output = np.zeros(decoderInputSize, dtype = "float32")

## populate the empty input and output vectors
inputText = list(df['speaker'])
outputText = list(df['response'])

for i, (i_t, o_t) in enumerate(zip(inputText, outputText)):
    #print(inputText[i])
    inputList = inputText[i].split(' ')
    outputList = outputText[i].split(' ')
    endInd = len(inputList)
    if endInd > maxLenPrompt - 1:
        endInd = maxLenPrompt - 1  
    for x in range(endInd):
        if inputList[x] in promptVocab:
            ind = promptVocab.index(inputList[x])
        else:
            ind = promptVocab.index("<OTHER>")
        encoder_input[i][x][ind] = 1.0
    encoder_input[i][x+1][promptVocab.index("<END>")] = 1.0
    endInd = len(outputList)
    if endInd > maxLenResponse - 1:
        endInd = maxLenResponse - 1
    for x in range(endInd):
        if outputList[x] in responseVocab:
            ind = responseVocab.index(outputList[x])
        else:
            ind = responseVocab.index("<OTHER>")
        decoder_input[i][x][ind] = 1.0
        if i >=1:
            decoder_output[i][x-1][ind] = 1.0
    decoder_input[i][x+1][responseVocab.index("<END>")] = 1.0
    decoder_output[i][x][responseVocab.index("<END>")] = 1.0

In [6]:
encoder_inputs = keras.Input(shape = (None, promptVocabSize))
encoder = keras.layers.LSTM(256, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape = (None, responseVocabSize))

decoder_lstm = keras.layers.LSTM(256, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = keras.layers.Dense(responseVocabSize, activation = "softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [7]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [encoder_input, decoder_input],
    decoder_output,
    batch_size=32,
    epochs=5
    #,    validation_split=0.2,
)

Epoch 1/5
305/305 [==============================] - 363s 1s/step - loss: 1.5900 - accuracy: 0.0520
Epoch 2/5
305/305 [==============================] - 359s 1s/step - loss: 1.5859 - accuracy: 0.0521
Epoch 3/5
305/305 [==============================] - 374s 1s/step - loss: 1.5842 - accuracy: 0.0521
Epoch 4/5
305/305 [==============================] - 383s 1s/step - loss: 1.5841 - accuracy: 0.0521
Epoch 5/5
305/305 [==============================] - 446s 1s/step - loss: 1.5834 - accuracy: 0.0521


In [8]:
model.save("seq2seq")

INFO:tensorflow:Assets written to: seq2seq\assets


INFO:tensorflow:Assets written to: seq2seq\assets


In [9]:
model = keras.models.load_model("seq2seq")

In [10]:
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(256,))
decoder_state_input_c = keras.Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [46]:
def decode(input_seq):
    states_val = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, responseVocabSize))
    target_seq[0, 0, 0] = 1.0 ## ignore the first word
    
    stop = False
    sent = ""
    while(stop == False):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_val)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = responseVocab[sampled_token_index]
        sent = sent + sampled_word + " "
        if sampled_word == '<END>' or len(sent.split(" ")) > maxLenResponse:
            stop = True
        target_seq = np.zeros((1, 1, responseVocabSize))
        target_seq[0, 0, sampled_token_index] = 1.0
        
        states_val = [h,c]
    return sent

In [47]:
def encode(text):
    encoder_input = np.zeros((1, maxLenPrompt, promptVocabSize), dtype = "float32")
    endInd = len(text.split(" "))
    if endInd > maxLenPrompt - 1:
        endInd = maxLenPrompt - 1  
    for x in range(endInd):
        if inputList[x] in promptVocab:
            ind = promptVocab.index(inputList[x])
        else:
            ind = promptVocab.index("<OTHER>")
        encoder_input[0][x][ind] = 1.0
    encoder_input[0][x+1][promptVocab.index("<END>")] = 1.0
    return encoder_input

In [ ]:
## Remove stage directions which are typically in parenthesis (like this)
def preprocess(text, isInput = True):
    
    ## Remove stage directions which are typically in parenthesis (like this)
    text = re.sub(r'\([^)]*\)', '', str(text))
    
    ## Remove capital letters
    text = text.lower()
    
    ## Remove contractions
    contractions = re.findall("[']", text)
    for i in contractions:
        text = text.replace(i,"")
        
    #replace multiple punctuation with single
    repeatpunctuations = re.findall('[.,!?]{2,}',text)
    for repeatpunctuation in repeatpunctuations:
        text = text.replace(repeatpunctuation, repeatpunctuation[0])
    
    ## add spaces between last word and punctuation
    puncs = re.findall('[.!?,]', text)
    for i in puncs:
        text = text.replace(i," "+i[0]+" ")
        
    ## remove all punctuation
    text = re.sub(r'[^\w\s]', '', text)
        
    ## remove multiple spaces and replace with just one
    text =  re.sub(' +', ' ', text)
    
    ## remove trailing and leading spaces
    text = text.strip()
    
    return text

In [51]:
def addToCsv(lastText, nText):
    df = pd.read_csv('dataset.csv')
    df.loc[len(df.index)] = [lastText, nText]
    df.to_csv('dataset'+ name +'.csv', index = False)

In [57]:
isOn = True
name = input("Hello my name is Sheldon enter your name (Enter 'help' to change responses when in the bot): ")
lastText = ''
while(isOn):
    text = input("")
    if(text == 'help'):
        nText = input("How should I have responded: ")
        addToCsv(lastText, nText, name)
    encoded = encode(preprocess(text))
    output = decode(encoded)
    print("Sheldon: " + output)
    lastText = text

Hello my name is Sheldon enter your name (Enter 'help' to change responses when in the bot): Arjun
Hello Sheldon
Sheldon: i believe leonard <END>
How are you
Sheldon: i do let <END>
